# Build the logbook

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : June 16th 2021
- Update : July 7th 2021


Works directory per directory (day by day)

CCIN2P3:

- works with kernels anaconda3_auxtel (with libradtran) a

In [152]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookscc/ana_summer2021',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.egg',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages/pyarrow-4.0.1-py3.8-linux-x86_64.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/coveralls-3.0.1-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/astroquery-0.4.2.dev0-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-package

In [153]:
# reconstructed / good /xpos /ypos
info={\
      'Cor_holo4_003_RG610_HD120144_20210706_000098.fits':(1,1,256,2130),\
      'Cor_holo4_003_RG610_HD120144_20210706_000099.fits':(1,1,256,2130),\
      'Cor_holo4_003_RG610_HD120144_20210706_000100.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000101.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000102.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000103.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000104.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000105.fits':(1,1,256,2300),\
      'Cor_holo4_003_RG610_HD120144_20210706_000106.fits':(1,1,256,2300)
      
     }
 

# Imports

In [154]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [155]:
from astropy.io import fits

In [156]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [157]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [158]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [159]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [160]:
dir_logbooks="logbooks"

In [161]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [162]:
FLAG_HOLO=True
home=os.getenv("HOME")


In [163]:
if FLAG_HOLO:
    image_path="holo"
    version="v2.0"
    logbook_csv=os.path.join(dir_logbooks,"auxtelholologbook_2021_07_06_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelholologbook_2021_07_06_" + version + ".xlsx")
else:
    image_path="ronchi170lpmm"
    version="v2.0"
    logbook_csv=os.path.join(dir_logbooks,"auxtelronchilogbook_2021_07_06_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelronchilogbook_2021_07_06_" + version + ".xlsx")

In [164]:
!ls /sps/lsst/groups/auxtel/data/2021/holo/localreduced

2021-06-08  2021-06-10	2021-07-06


In [165]:
#dir_images="/Users/dagoret/DATA/AuxTelData2021/holo/FlipCleans"
# laptop
#dir_images=os.path.join(home,"DATA/AuxTelData2021/")+image_path+"/FlipCleans"
# CC
#dir_images="/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans"
dir_images="/sps/lsst/groups/auxtel/data/2021/holo/localreduced/2021-07-06"

In [166]:
all_files=os.listdir(dir_images)

In [167]:
all_files

['Cor_holo4_003_RG610_HD120144_20210706_000101.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000103.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000100.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000106.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000102.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000105.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000098.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000104.fits',
 'Cor_holo4_003_RG610_HD120144_20210706_000099.fits']

In [168]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [169]:
all_selected_files=np.array(all_selected_files)

In [170]:
#print(all_selected_files)

In [171]:
re.findall('^Cor_holo4_003_.*_(.*)[.]fits$','Cor_holo4_003_empty_HD115169_20210608_000291.fits')

['000291']

### Sort files

In [172]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('^Cor_holo4_003_.*_(.*)[.]fits$',filename)
    
    return m[0]

In [173]:
file_tag_forsorting('Cor_holo4_003_empty_HD115169_20210608_000291.fits')

'000291'

In [174]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['000101', '000103', '000100', '000106', '000102', '000105',
       '000098', '000104', '000099'], dtype='<U6')

In [175]:
indexes=np.argsort(all_tags)

In [176]:
sorted_files=all_selected_files[indexes]

In [177]:
sorted_files

array(['Cor_holo4_003_RG610_HD120144_20210706_000098.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000099.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000100.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000101.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000102.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000103.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000104.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000105.fits',
       'Cor_holo4_003_RG610_HD120144_20210706_000106.fits'], dtype='<U49')

In [178]:
N=len(sorted_files)

### Filter bad files

In [179]:
idx=0
indexes_bad=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0['OBJECT'],"\t",filename)
    if hdr0['OBJECT'] == '35 Tau' or hdr0['OBJECT'] == 'Park position':
        print('bad file ', filename)
        indexes_bad.append(idx)
        
    idx+=1

In [180]:
indexes_bad

[]

### Remove bad files

In [181]:
sorted_files=np.delete(sorted_files,indexes_bad)

### Check for target

In [182]:
re.findall('^Cor_holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*[.]fits$','Cor_holo4_003_RG610_HD120144_20210706_000098.fits')

['HD120144']

In [183]:
def file_target(filename):
    m=re.findall('^Cor_holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*[.]fits$',filename)
    if len(m)>0:
        return m
    elif re.search('NGC4755',filename):
        return ['NGC4755']
    else:
        return []

In [184]:
for filename in sorted_files:
    print(file_target(filename),"\t",filename)

['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000098.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000099.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000100.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000101.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000102.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000103.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000104.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000105.fits
['HD120144'] 	 Cor_holo4_003_RG610_HD120144_20210706_000106.fits


In [185]:
idx=0
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0,"\t",filename)
        
    idx+=1

# Build the logbook from the headers

In [186]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=file_target(filename)[0]
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [187]:
df=pd.DataFrame()

In [188]:
len(all_airmass)

9

In [189]:
N=len(all_indexes)

## Fill with container info

In [190]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [191]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [192]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [193]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [194]:
pd.set_option('display.max_rows', None)

In [195]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2021-07-07T01:16:33.033,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000098.fits,98,HD120144,RG610,holo4_003,1.112792,20.0,...,7.0,16.0,11.6,256.0,2130.0,NaN,NaN,,1.0,1.0
1,1,2021-07-07T01:17:01.054,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000099.fits,99,HD120144,RG610,holo4_003,1.113402,20.0,...,7.0,16.0,11.6,256.0,2130.0,NaN,NaN,,1.0,1.0
2,2,2021-07-07T01:17:25.525,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000100.fits,100,HD120144,RG610,holo4_003,1.113971,20.0,...,7.0,16.0,11.6,256.0,2300.0,NaN,NaN,,1.0,1.0
3,3,2021-07-07T01:17:50.010,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000101.fits,101,HD120144,RG610,holo4_003,1.114548,20.0,...,7.0,15.0,13.1,256.0,2300.0,NaN,NaN,,1.0,1.0
4,4,2021-07-07T01:18:14.601,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000102.fits,102,HD120144,RG610,holo4_003,1.115124,20.0,...,7.0,15.0,13.1,256.0,2300.0,NaN,NaN,,1.0,1.0
5,5,2021-07-07T01:18:38.991,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000103.fits,103,HD120144,RG610,holo4_003,1.115701,20.0,...,7.1,15.0,10.9,256.0,2300.0,NaN,NaN,,1.0,1.0
6,6,2021-07-07T01:19:03.593,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000104.fits,104,HD120144,RG610,holo4_003,1.116286,20.0,...,7.1,15.0,10.9,256.0,2300.0,NaN,NaN,,1.0,1.0
7,7,2021-07-07T01:19:27.983,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000105.fits,105,HD120144,RG610,holo4_003,1.116869,20.0,...,7.1,15.0,10.9,256.0,2300.0,NaN,NaN,,1.0,1.0
8,8,2021-07-07T01:19:52.570,/sps/lsst/groups/auxtel/data/2021/holo/localre...,Cor_holo4_003_RG610_HD120144_20210706_000106.fits,106,HD120144,RG610,holo4_003,1.117460,20.0,...,7.2,14.0,9.9,256.0,2300.0,NaN,NaN,,1.0,1.0


# Make logbook


In [196]:
df.to_excel(logbook_xlsx)
df.to_csv(logbook_csv)